In [1]:
import pandas as pd

# nama file adalah 'scrapping_link_kompas_bpjs.csv'
df_links = pd.read_csv("C:/Users/User/Jupyter Notebook/PBA/Bahan Kelompok/Scrapping News Text/scraping_cnn_bpjs_gaya-hidup.csv")

# Tampilkan beberapa baris pertama untuk verifikasi
df_links.head()


,Judul,Link,Kategori,Tanggal Upload
0,"Bantu Donasi untuk Pengobatan Jafar, Bocah den...",https://www.cnnindonesia.com/gaya-hidup/202406...,Gaya Hidup,3 bulan yang lalu
1,Menimbang Untung Rugi Asuransi yang Tak Pernah...,https://www.cnnindonesia.com/gaya-hidup/202403...,Gaya Hidup,5 bulan yang lalu
2,Daikin Jadi Korporasi Teratas Dalam Kepedulian...,https://www.cnnindonesia.com/gaya-hidup/202403...,Gaya Hidup,5 bulan yang lalu
3,Mayapada dan Kalbe Farma Wujudkan Ekosistem La...,https://www.cnnindonesia.com/gaya-hidup/202402...,Gaya Hidup,7 bulan yang lalu
4,"Cabut Omnibus Law, Agenda Pertama Partai Buruh...",https://www.cnnindonesia.com/gaya-hidup/202402...,Gaya Hidup,7 bulan yang lalu


In [2]:
df_links = df_links.dropna()

In [3]:
df_links.head()

,Judul,Link,Kategori,Tanggal Upload
0,"Bantu Donasi untuk Pengobatan Jafar, Bocah den...",https://www.cnnindonesia.com/gaya-hidup/202406...,Gaya Hidup,3 bulan yang lalu
1,Menimbang Untung Rugi Asuransi yang Tak Pernah...,https://www.cnnindonesia.com/gaya-hidup/202403...,Gaya Hidup,5 bulan yang lalu
2,Daikin Jadi Korporasi Teratas Dalam Kepedulian...,https://www.cnnindonesia.com/gaya-hidup/202403...,Gaya Hidup,5 bulan yang lalu
3,Mayapada dan Kalbe Farma Wujudkan Ekosistem La...,https://www.cnnindonesia.com/gaya-hidup/202402...,Gaya Hidup,7 bulan yang lalu
4,"Cabut Omnibus Law, Agenda Pertama Partai Buruh...",https://www.cnnindonesia.com/gaya-hidup/202402...,Gaya Hidup,7 bulan yang lalu


In [4]:
pip install requests beautifulsoup4

In [4]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import json
import pandas as pd

def scrape_cnn(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    judul = soup.find('h1', class_='mb-2').text.strip() if soup.find('h1', class_='mb-2') else "N/A"
    
    # Mengambil tag dari script JSON-LD
    tag = "N/A"
    scripts = soup.find_all('script', type='application/ld+json')
    for script in scripts:
        data = json.loads(script.string)
        if data.get("@type") == "BreadcrumbList":
            items = data.get("itemListElement", [])
            if len(items) > 2:
                tag = items[2].get("name", "N/A")
                break
    
    link = url
    
    content_element = soup.find('div', class_='detail-text')
    text = ' '.join([p.text for p in content_element.find_all('p')]) if content_element else "N/A"
    
    # Mengambil date dari script JSON-LD
    date = "N/A"
    for script in scripts:
        data = json.loads(script.string)
        if data.get("@type") == "WebPage":
            date = data.get("datePublished", "N/A")
            break
    
    return {
        'Judul': judul,
        'Tag': tag,
        'Link': link,
        'Text': text,
        'Date': date
    }

def clean_text(text):
    cleaned_text = text.replace('\n', ' ').replace('\r', '').replace('\t', ' ')
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

def main():
    # Membaca file CSV yang berisi link berita
    df = pd.read_csv('C:/Users/User/Jupyter Notebook/PBA/Bahan Kelompok/Scrapping News Text/scraping_cnn_bpjs_gaya-hidup.csv')
    links = df['Link'].tolist()
    
    results = []
    
    for link in links:
        try:
            result = scrape_cnn(link)
            result['Text'] = clean_text(result['Text'])
            results.append(result)
            print(f"Berhasil scraping: {link}")
        except Exception as e:
            print(f"Gagal scraping {link}: {str(e)}")
    
    # Menyimpan hasil ke file CSV
    filename = f'cnn_news_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Judul', 'Tag', 'Link', 'Text', 'Date']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for result in results:
            writer.writerow(result)
    
    print(f'Scraping selesai. Hasil disimpan dalam file {filename}')

if __name__ == '__main__':
    main()

Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20240610122410-289-1107965/bantu-donasi-untuk-pengobatan-jafar-bocah-dengan-kelainan-usus-langka
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20240326140725-307-1079143/menimbang-untung-rugi-asuransi-yang-tak-pernah-diklaim
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20240326153013-307-1079209/daikin-jadi-korporasi-teratas-dalam-kepedulian-terhadap-pekerja-rentan
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20240215180352-260-1063177/mayapada-dan-kalbe-farma-wujudkan-ekosistem-layanan-kanker-terpadu
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20240209112401-307-1060384/cabut-omnibus-law-agenda-pertama-partai-buruh-di-senayan
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20240207110608-307-1059676/kampanye-di-papua-partai-buruh-siap-kerja-buat-rakyat
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20240130140022-255-1056196/melihat-lebih-detail-vis

Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20191125192410-255-451418/kondisi-orang-yang-perlu-pasang-ring-jantung
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20191123133546-255-450857/defisit-bpjs-menkes-minta-rs-kurangi-sesar-dan-pasang-ring
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20191112195313-260-447781/supriyanto-dharmoredjo-sosok-di-balik-rsud-terbaik-di-dunia
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20191030181210-255-444241/atasi-penyakit-tidak-menularmenkes-terawan-andalkan-germas
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20191030152244-255-444171/catatan-masalah-kesehatan-dari-idi-untuk-menkes-terawan
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20191030114921-255-444078/sempat-renggang-idi-dan-menkes-terawan-kembali-mesra
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20191030102457-255-444048/menkes-terawan-datangi-kantor-pb-idi-dan-gelar-rapat-tertutup
Berhasil sc

Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20160623125239-255-140375/kejadian-bunuh-diri-tak-dianggap-serius-di-indonesia
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20160531043412-255-134602/penyakit-akibat-rokok-disebut-rugikan-negara-us-45-triliun
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20160524200938-255-133197/langkah-panjang-sehatkan-penduduk-indonesia
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20160212112048-255-110521/rumah-singgah-penyelamat-untuk-pasien-kanker-anak
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20160212100858-255-110493/kumpulan-receh-bernilai-miliaran-untuk-bocah-penderita-kanker
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20160204082016-255-108766/menduga-nasib-pasar-kesehatan-indonesia-2016
Berhasil scraping: https://www.cnnindonesia.com/gaya-hidup/20151228095923-255-100666/kisah-kanya-puspokusumo-berdamai-dengan-multipel-sklerosis
Berhasil scraping: https://www.cn